In [20]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd
import codecs
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
import csv
import string
from datetime import datetime
from collections import Counter
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import   RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
# --------------------------------------- #
import pyarabic.araby as araby
import pyarabic.number as number
from pyarabic.araby import strip_tashkeel
from pyarabic.araby import strip_tatweel
from pyarabic.araby import tokenize, is_arabicrange, strip_tashkeel
# --------------------------------------- #
import pyarabic.arabrepr
arepr = pyarabic.arabrepr.ArabicRepr()
repr = arepr.repr
from tashaphyne.stemming import ArabicLightStemmer
import emoji


import seaborn
import matplotlib.pyplot as plt

%matplotlib inline

[nltk_data] Downloading package punkt to /Users/nadaa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [21]:
df = pd.read_csv('/Users/nadaa/Desktop/Osama/Testingg.csv')

In [22]:
newdata = df.reset_index()
newdata["id"] = newdata.index 
#newdata.head()
newdata.columns= newdata.columns.str.lower()
newdata.columns

Index(['level_0', 'unnamed: 0', 'index', 'screen_name', 'text', 'sentiment',
       'id', 'predictions'],
      dtype='object')

In [23]:
newdata.drop(['index'], axis=1  , inplace=True)

In [24]:
new = newdata[["screen_name",'text','id']] 
new.head()

,screen_name,text,id
0,M_World2020,"['بلاك', 'فرايدي', 'شي', 'إن', 'جامعة', 'الحفر...",0
1,B7Kgsvd72R2jzNL,"['جمعنا', 'لكم', 'أهم', 'النصائح', 'لتقليل', '...",1
2,Abohedab1,"['الجار', 'قبل', 'الدار', 'والأخضر', 'قبل', 'ا...",2
3,Hanoma29441020,"['جمعنا', 'لكم', 'أهم', 'النصائح', 'لتقليل', '...",3
4,560Mohammad,"['بلاك', 'فرايدي', 'شي', 'إن', 'جامعة', 'الحفر...",4


In [25]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

# def space(text):
#     text= re.sub(r"_", '\s+', s)
#     return text

# new1['text'].replace('_', ' ', regex=True, inplace=True)


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_hashtag(text):
    return text.replace("#\\p{IsAlphabetic}+", "");

In [26]:
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)
    
    #Replace RT with empty string
    tweet = re.sub('RT', ' ', tweet)
    
    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)
    
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    
    
    #remove non-arabic
#      tweet = re.sub(r'[^ء-ي0-9]',' ',tweet)

    return tweet

In [27]:
new['text'].replace('[^ء-ي0-9]', ' ', regex=True, inplace=True)

new['text'].replace('_', ' ', regex=True, inplace=True)

new['text'] = new['text'].str.replace(r'\d+','')

new['text']= new['text'].apply(lambda x: remove_diacritics(x))

new['text']= new['text'].apply(lambda x: remove_repeating_char(x))

new['text']= new['text'].apply(lambda x: processPost(x))

new['text']= new['text'].apply(lambda x: remove_hashtag(x))

new['text']= new['text'].apply(lambda x: remove_punctuations(x))

new['text']= new['text'].apply(lambda x: remove_repeating_char(x))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
<ipython-input-27-59b73772f412>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['text'] = new['text'].str.replace(r'\d+','')
<ipython-input-27-59b73772f412>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [28]:
tokens = TweetTokenizer()
new['text'] = new['text'].apply(tokens.tokenize)

<ipython-input-28-3ad1ae8a2869>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['text'] = new['text'].apply(tokens.tokenize)


In [29]:
new.shape

(20987, 3)

In [30]:
stop = ['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين','و', 'بيد',
         'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك',
         'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما',
         'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك',
         'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا',
         'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا',
         'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير',
         'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها',
         'قد', 'كأن', 'كأنما', 'كأي', 'كأين', 'كذا', 'كذلك', 'كل', 'كلا',
         'كلاهما', 'كلتا', 'كلما', 'كليكما', 'كليهما', 'كم', 'كم', 'كما',
         'كي', 'كيت', 'كيف', 'كيفما', 'لا', 'لاسيما', 'لدى', 'لست', 'لستم',
         'لستما', 'لستن', 'لسن', 'لسنا', 'لعل', 'لك', 'لكم', 'لكما',
         'لكن', 'لكنما', 'لكي', 'لكيلا', 'لم', 'لما', 'لن', 'لنا',
         'له', 'لها', 'لهم', 'لهما', 'لهن', 'لو', 'لولا', 'لوما',
         'لي', 'لئن', 'ليت', 'ليس', 'ليسا', 'ليست', 'ليستا', 'ليسوا', 'ما',
         'ماذا', 'متى', 'مذ', 'مع', 'مما', 'ممن', 'من', 'منه', 'منها', 'منذ',
         'مه', 'مهما', 'نحن', 'نحو', 'نعم', 'ها', 'هاتان', 'هاته', 'هاتي',
         'هاتين', 'هاك', 'هاهنا', 'هذا', 'هذان', 'هذه', 'هذي', 'هذين', 'هكذا',
         'هل', 'هلا', 'هم', 'هما', 'هن', 'هنا', 'هناك', 'هنالك', 'هو', 'هؤلاء',
         'هي', 'هيا', 'هيت', 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن',
         'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا' , 'من' , 'على', 'الى','هما', 'مع', 'هذه', 'التي', 'كما ', 'ذلك ', 'لذا', 'عن', 'في','ان','كان','كانت','الى','قبل','أنه','تم'
        ,'وقال','قال','فى','وقد','قد','ولم','وذلك','ذلك','يكون','او','وهذه','وهي ','وبعد','وهذا','عندها','جدا','بأن','انه','الي']

In [31]:
new['text'] = [w for w in new['text'] if not w in stop]
new.head()

<ipython-input-31-4cdd638f4f34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['text'] = [w for w in new['text'] if not w in stop]


,screen_name,text,id
0,M_World2020,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",0
1,B7Kgsvd72R2jzNL,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",1
2,Abohedab1,"[الجار, قبل, الدار, والأخضر, قبل, الأحمر, وفرت...",2
3,Hanoma29441020,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",3
4,560Mohammad,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",4


In [32]:
y = new['text'].apply(lambda x: ' '.join(x))
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=250, min_df=5, max_df=0.7, stop_words=stop)
X = vectorizer.fit_transform(y).toarray()

/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['وهي'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [33]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

In [34]:
import pickle
# acc = 77 or 79
with open('RFModel250', 'rb') as training_model:
    taqa = pickle.load(training_model)

In [35]:
new['predictions'] = taqa.predict(X)
#Or newdata['predictions'] = pd.DataFrame(Predictions) - if Length of values does not match length of index

<ipython-input-35-a35aa5db218b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['predictions'] = taqa.predict(X)


In [36]:
new.head()

,screen_name,text,id,predictions
0,M_World2020,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",0,Neutral
1,B7Kgsvd72R2jzNL,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",1,Positive
2,Abohedab1,"[الجار, قبل, الدار, والأخضر, قبل, الأحمر, وفرت...",2,Positive
3,Hanoma29441020,"[جمعنا, لكم, أهم, النصائح, لتقليل, من, استهلاك...",3,Positive
4,560Mohammad,"[بلاك, فرايدي, شي, إن, جامعة, الحفر, عن, بعد, ...",4,Neutral
